## Count image Numbers ##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

def count_images_in_folder(folder_path):
    # List all files in the folder
    all_files = os.listdir(folder_path)

    # Filter image files
    image_files = [file for file in all_files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Return the count of image files
    return len(image_files)

# Specify the paths to the training and testing folders
train_folder_path = '/content/drive/MyDrive/Fishtrain/Tail And Fin Rot'
test_folder_path = '/content/drive/MyDrive/Fishtrain/THE BACTERIAL GILL ROT'

# Call the function to get the count for training and testing sets
train_image_count = count_images_in_folder(train_folder_path)
test_image_count = count_images_in_folder(test_folder_path)

# Print the results
print(f'The number of images in the training set is: {train_image_count}')
print(f'The number of images in the testing set is: {test_image_count}')


The number of images in the training set is: 705
The number of images in the testing set is: 705


## Augment Img ##

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np

def augment_images(input_folder, output_folder, target_image_count):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Initialize ImageDataGenerator for data augmentation
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Get a list of all image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Calculate the number of augmentations needed per image to reach the target count
    augmentations_per_image = int(np.ceil(target_image_count / len(image_files)))

    for image_file in image_files:
        # Load the image
        img_path = os.path.join(input_folder, image_file)
        img = image.load_img(img_path)

        # Convert the image to a Numpy array with shape (1, height, width, channels)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Generate augmented images and save them to the output folder
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=output_folder, save_prefix=image_file.split('.')[0], save_format='jpeg'):
            i += 1
            if i >= augmentations_per_image:
                break

# Specify the paths to input and output folders
input_folder = '/content/drive/MyDrive/dataset/Fishtrain/Argulus'  ## specific folder path not a folder contains subfolder
output_folder = '/content/drive/MyDrive/dataset/Fishtrain/Argulus/augment'
target_image_count = 500  # Set the target number of images per folder

# Call the function to augment images in each folder
augment_images(input_folder, output_folder, target_image_count)


## Copy img from one file to another ##

In [ ]:
import os
import shutil
import random

def copy_random_images(source_folder, destination_folder, num_images_to_copy=500):
    # List all files in the source folder
    all_files = os.listdir(source_folder)

    # Filter image files
    image_files = [file for file in all_files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Ensure that the number of images to copy does not exceed the total number of images
    num_images_to_copy = min(num_images_to_copy, len(image_files))

    # Randomly select images
    selected_images = random.sample(image_files, num_images_to_copy)

    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Copy selected images to the destination folder
    for image_file in selected_images:
        source_path = os.path.join(source_folder, image_file)
        destination_path = os.path.join(destination_folder, image_file)
        shutil.copyfile(source_path, destination_path)

# Specify the paths to the source and destination folders
source_folder = '/content/drive/MyDrive/archive/Augment_Output_Images/4'
destination_folder = '/content/drive/MyDrive/archive/Augment_Images/4'

# Call the function to copy 500 random images
copy_random_images(source_folder, destination_folder, num_images_to_copy=500)


## Gray to pseudocolor##

In [ ]:
import os
import cv2

def convert_grayscale_to_pseudocolor(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    file_list = os.listdir(input_folder)

    for file_name in file_list:
        # Construct the input and output file paths
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        # Read the grayscale image
        grayscale_img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

        if grayscale_img is not None:
            # Normalize pixel values to the range [0, 255]
            normalized_img = cv2.normalize(grayscale_img, None, 0, 255, cv2.NORM_MINMAX)

            # Apply a pseudocolor map ('jet' colormap)
            pseudocolor_img = cv2.applyColorMap(normalized_img, cv2.COLORMAP_JET)

            # Save the pseudocolor image
            cv2.imwrite(output_path, pseudocolor_img)
            print(f"Converted: {file_name}")
        else:
            print(f"Failed to read: {file_name}")

if __name__ == "__main__":
    # Specify the input and output folders
    input_folder_path = "/content/drive/MyDrive/archive/Augment_Images"
    output_folder_path = "/content/drive/MyDrive/archive/Psedocolor_Images"

    # Convert grayscale images to pseudocolor
    convert_grayscale_to_pseudocolor(input_folder_path, output_folder_path)


In [ ]:
import os
import cv2

def convert_grayscale_to_pseudocolor(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files and subdirectories in the input folder
    for root, dirs, files in os.walk(input_folder):
        for file_name in files:
            # Construct the input and output file paths
            input_path = os.path.join(root, file_name)
            relative_path = os.path.relpath(input_path, input_folder)
            output_path = os.path.join(output_folder, relative_path)

            # Read the grayscale image
            grayscale_img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

            if grayscale_img is not None:
                # Apply a pseudocolor map
                pseudocolor_img = cv2.applyColorMap(grayscale_img, cv2.COLORMAP_JET)

                # Save the pseudocolor image
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                cv2.imwrite(output_path, pseudocolor_img)
                print(f"Converted: {relative_path}")
            else:
                print(f"Failed to read: {relative_path}")

if __name__ == "__main__":
    # Specify the input and output folders
    input_folder_path = "/content/drive/MyDrive/archive/Augment_Images"
    output_folder_path = "/content/drive/MyDrive/archive/Psedocolor_Images"

    # Convert grayscale images to pseudocolor recursively
    convert_grayscale_to_pseudocolor(input_folder_path, output_folder_path)


##Test_Train_Split ##


In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil

def train_test_split_images(input_folder, output_train_folder, output_test_folder, test_size=0.2, random_seed=42):
    # List all files in the input folder
    all_files = os.listdir(input_folder)

    # Filter image files
    image_files = [file for file in all_files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Use train_test_split to split the images into training and testing sets
    train_images, test_images = train_test_split(image_files, test_size=test_size, random_state=random_seed)

    # Create output folders if they don't exist
    os.makedirs(output_train_folder, exist_ok=True)
    os.makedirs(output_test_folder, exist_ok=True)

    # Copy training images to the output train folder
    for image_file in train_images:
        source_path = os.path.join(input_folder, image_file)
        destination_path = os.path.join(output_train_folder, image_file)
        shutil.copyfile(source_path, destination_path)

    # Copy testing images to the output test folder
    for image_file in test_images:
        source_path = os.path.join(input_folder, image_file)
        destination_path = os.path.join(output_test_folder, image_file)
        shutil.copyfile(source_path, destination_path)

# Specify the paths to the input, output train, and output test folders
input_folder = '/content/drive/MyDrive/archive/Psedocolor_Images/4'
output_train_folder = '/content/drive/MyDrive/archive/Psedo_Test_Train/Train/4'
output_test_folder = '/content/drive/MyDrive/archive/Psedo_Test_Train/Test/4'

# Call the function to perform the train-test split
train_test_split_images(input_folder, output_train_folder, output_test_folder, test_size=0.2, random_seed=42)


In [ ]:
import os
import shutil
import random

def split_data(input_folder, output_folder, train_ratio=0.8, val_ratio=0.2, test_ratio=0.1, seed=42):
    # Create output folders if they don't exist
    for folder in ['train', 'validation', 'test']:
        os.makedirs(os.path.join(output_folder, folder), exist_ok=True)

    # Set a random seed for reproducibility
    random.seed(seed)

    # Get the list of image files
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Shuffle the list of image files
    random.shuffle(image_files)

    # Calculate the number of images for each split
    total_images = len(image_files)
    num_train = int(train_ratio * total_images)
    num_val = int(val_ratio * total_images)

    # Assign images to each split
    train_images = image_files[:num_train]
    val_images = image_files[num_train:num_train + num_val]
    test_images = image_files[num_train + num_val:]

    # Copy images to their respective folders
    for image in train_images:
        shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, 'train', image))

    for image in val_images:
        shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, 'validation', image))

    for image in test_images:
        shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, 'test', image))

# Replace 'input_folder' and 'output_folder' with your actual paths
input_folder = '/content/drive/MyDrive/Augment_Train/Benign'
output_folder = '/content/drive/MyDrive/Spilited_Augment/Benign'

split_data(input_folder, output_folder)

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

def train_test_split_with_validation(input_folder, output_folder, test_size=0.2, validation_size=0.1, random_state=42):
    # Create output folders if they don't exist
    for folder in ['train', 'validation', 'test']:
        os.makedirs(os.path.join(output_folder, folder), exist_ok=True)

    # Get the list of image files
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Split into initial training and test sets
    train_files, test_files = train_test_split(image_files, test_size=test_size, random_state=random_state)

    # Further split the training set into training and validation sets
    train_files, val_files = train_test_split(train_files, test_size=validation_size/(1 - test_size), random_state=random_state)

    # Copy images to their respective folders
    for image in train_files:
        shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, 'train', image))

    for image in val_files:
        shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, 'validation', image))

    for image in test_files:
        shutil.copy(os.path.join(input_folder, image), os.path.join(output_folder, 'test', image))

# Example usage
# Replace '/path/to/your/images' and '/path/to/output/folder' with actual paths
input_folder = '/path/to/your/images'
output_folder = '/path/to/output/folder'

train_test_split_with_validation(input_folder, output_folder)
